# Making predictions with Prophet on IBM Watson Machine Learning

<div>
<img src="https://miro.medium.com/max/1400/1*PRu54C1ggbvV2-InBOE0xw.jpeg" width="500"/>
</div>


In this notebook, I demonstrate a simple example of how you can make predictions with Prophet on Watson Machine Learning (WML).

As WML does not natively support Prophet, we will need to create a wrapper function which allows us to install custom packages.

As discussed in my post, we are not able to install Prophet directly in WML environment thus, I have created the [Prophet Inference](https://github.com/randyphoa/prophet-inference) package which is a light weight inference-only version of Prophet.

I will be extending the [simple example](https://facebook.github.io/prophet/docs/quick_start.html) from Prophet based on the [Peyton Manning dataset](https://github.com/facebook/prophet/blob/master/examples/example_wp_log_peyton_manning.csv).

More details can be found at at [Making predictions with Prophet on IBM Watson Machine Learning](https://medium.com/@randyphoa/making-predictions-with-prophet-on-ibm-watson-machine-learning-abc42ca129f3). 

## Install required packages
Refer to https://facebook.github.io/prophet/docs/installation.html if you encounter issues installing fbprophet.


In [ ]:
! pip install fbprophet fbprophet-inference 

## Import packages

In [ ]:
import io
import os
import sys
import json
import boto3
import pickle
import datetime
import pandas as pd
from fbprophet import Prophet
import fbprophet_inference
from fbprophet_inference import ProphetInference, serialize
from watson_machine_learning_client import WatsonMachineLearningAPIClient

client_cos = boto3.client("s3", endpoint_url="x", aws_access_key_id="x", aws_secret_access_key="x", region_name="x")

def read(key):
    return client_cos.get_object(Bucket="cloud-object-storage-cpd-data", Key=key)["Body"].read()

def write(obj, key):
    client_cos.put_object(Bucket="cloud-object-storage-cpd-data", Key=key, Body=io.StringIO(model_json).getvalue())
    
wml_credentials = {
    "token": os.environ["USER_ACCESS_TOKEN"],
    "instance_id" : "wml_local",
    "url": os.environ["RUNTIME_ENV_APSX_URL"],
    "version": "2.5.0"
}

def guid_from_space_name(client, space_name):
    instance_details = client.service_instance.get_details()
    space = client.spaces.get_details()
    return(next(item for item in space["resources"] if item["entity"]["name"] == space_name)["metadata"]["guid"])

client = WatsonMachineLearningAPIClient(wml_credentials)
space_uid = guid_from_space_name(client, "Deployment Space")
client.set.default_space(space_uid)

## Fit model on historical data

In [3]:
df = pd.read_csv("https://raw.githubusercontent.com/facebook/prophet/master/examples/example_wp_log_peyton_manning.csv")
m = Prophet()
m.fit(df)

### Last date in dataset

In [4]:
df.tail(1)

,ds,y
2904,2016-01-20,8.891374


## Predict 2 weeks ahead from 2016-01-21 to 2016-02-02

In [5]:
df_pred = pd.date_range(start="2016-01-21", periods=14, freq="D").to_frame(name="ds")
prophet_pred = m.predict(df_pred)[["ds", "yhat"]]

## Export model parameters to json
Model parameters are estimated from the fit function that was called earlier.

Save exported model to Cloud Object Storage.

In [6]:
model_json = fbprophet_inference.serialize.model_to_json(m)
write(model_json, "model_json.json")
# list(dict(json.loads(model_json)).keys())

## Create function to be deployed to WML

The below function creates a wrapper around the score function where we can install custom packages,

In this example, I have exported my model parameters to Cloud Object Store. Other options are storing it as an asset within Cloud Pak for Data or downloading from an external location like GitHub if the model parameters are not confidential.

In [7]:
def func():
    import subprocess
    subprocess.check_output("pip install fbprophet-inference --user", stderr=subprocess.PIPE, shell=True)
    subprocess.check_output("pip install boto3 --user", stderr=subprocess.PIPE, shell=True)
    
    import io
    import json
    import boto3
    import pickle
    import datetime
    import pandas as pd
    import fbprophet_inference
    from fbprophet_inference import ProphetInference, serialize

    json.JSONEncoder.default = lambda self,obj: (obj.isoformat() if isinstance(obj, datetime.datetime) else None)    
    client_cos = boto3.client("s3", endpoint_url="x", aws_access_key_id="x", aws_secret_access_key="x", region_name="ap-geo")
    
    model_json = client_cos.get_object(Bucket="cloud-object-storage-cpd-data", Key="model_json.json")["Body"].read().decode("utf_8")
    model = fbprophet_inference.serialize.model_from_json(model_json)
    
    def score(payload):
        date_from, date_to = payload["input_data"][0]["values"][0]
        df_pred = pd.date_range(start=date_from, end=date_to, freq="D").to_frame(name="ds")
        pred = model.predict(df_pred)[["ds", "yhat"]]
        return {"predictions": [ {"result": [pred.to_dict()]} ]}
    return score
    
# input_data = {"input_data": [{"fields": ["DATE_FROM", "DATE_TO"], "values": [["2016-01-21", "2016-02-02"]]}]}
# print(func()(input_data))

## Deploy function to WML

In [8]:
FUNCTION_NAME = "function-prophet-powersaving"
DEPLOYMENT_FUNCTION_NAME = "function-prophet-deploy"

for deployment in client.deployments.get_details()["resources"]:
    if deployment["entity"]["name"] == DEPLOYMENT_FUNCTION_NAME:
        print(f"Deleting deployment: {DEPLOYMENT_FUNCTION_NAME}")
        print(client.deployments.delete(deployment["metadata"]["guid"]))
        print()

for function_detail in client.repository.get_function_details()["resources"]:
    if function_detail["entity"]["name"] == FUNCTION_NAME:
        print(f"Deleting function: {FUNCTION_NAME}")
        print(client.repository.delete(function_detail["metadata"]["guid"]))
        print()
        
meta_data = { 
    client.repository.FunctionMetaNames.NAME : FUNCTION_NAME,
    client.repository.FunctionMetaNames.RUNTIME_UID: "ai-function_0.1-py3.6"
}

function_details = client.repository.store_function(meta_props=meta_data, function=func)

function_uid = client.repository.get_function_uid(function_details)

meta_props = {
   client.deployments.ConfigurationMetaNames.NAME: DEPLOYMENT_FUNCTION_NAME,
   client.deployments.ConfigurationMetaNames.ONLINE: {}
}
deployment_details = client.deployments.create(function_uid, meta_props=meta_props)
deployment_uid = client.deployments.get_uid(deployment_details)

print(deployment_uid)



#######################################################################################

Synchronous deployment creation for uid: 'ff1b698f-e7ee-42ac-b895-f4f4a59e47f5' started

#######################################################################################


initializing...
ready


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='dbb7ddb3-f816-446b-802f-2428db193182'
------------------------------------------------------------------------------------------------


dbb7ddb3-f816-446b-802f-2428db193182


## Test function

In [12]:
scoring_payload = {
   client.deployments.ScoringMetaNames.INPUT_DATA: [{
        "fields": ["DATE_FROM", "DATE_TO"],
        "values": [["2016-01-21", "2016-02-03"]]
    }]
}

predictions = client.deployments.score(deployment_uid, scoring_payload)
prophet_inference_pred = pd.read_json(json.dumps(predictions["predictions"][0]["result"][0])).sort_index()

## Ensure results are the same between Prophet and Prophet Inference

In [10]:
prophet_pred

,ds,yhat
0,2016-01-21,8.554653
1,2016-01-22,8.570631
2,2016-01-23,8.339292
3,2016-01-24,8.707247
4,2016-01-25,9.015927
5,2016-01-26,8.784269
6,2016-01-27,8.594901
7,2016-01-28,8.581662
8,2016-01-29,8.571898
9,2016-01-30,8.312057


In [13]:
prophet_inference_pred

,ds,yhat
0,2016-01-21T00:00:00,8.554653
1,2016-01-22T00:00:00,8.570631
2,2016-01-23T00:00:00,8.339292
3,2016-01-24T00:00:00,8.707247
4,2016-01-25T00:00:00,9.015927
5,2016-01-26T00:00:00,8.784269
6,2016-01-27T00:00:00,8.594901
7,2016-01-28T00:00:00,8.581662
8,2016-01-29T00:00:00,8.571898
9,2016-01-30T00:00:00,8.312057
